#**TAIL-GPT2**
##*Diretoria de Processamento de Linguagem Natural*
####*Autor: Felipe Honorato*

Bom, nesse notebook iremos entender o funcionamento da GPT2 e adaptaremos ela para gerar analises do trip advisor. 

A GPT-2 é construída em cima dos transformers, um tipo de rede que utiliza um mecanísmo de atenção para guiar a rede em onde focar durante a predição da próxima palavra.

No paper attention is all you need voces podem ver mais detalhes sobre esse mecanismo. [Attention is all you need](https://arxiv.org/abs/1706.03762)

A GPT-2 é uma rede treinada com corpus bem vasto extraído da internet. O modelo possui versões menores que nos possibilitam treinar pelo colab. Utilizaremos a implementação da biblioteca [Huggingface](https://huggingface.co/). Lá voces tem acesso acesso à documentação e a um forum repleto de tutoriais.

**PS: Lembre de usar GPU nesse notebook**

***
# **Preparando o ambiente**

#### Instalando bibliotecas e baixando dataset

In [ ]:
%%capture
!pip install transformers #Huggingface
!gdown --id 15_qLjo8_qgTAIWqVWpp_JMKzfpwQT29B #Baixando o dataset (Fiz upload no meu drive)
!pip install datasets

#### Importação das bibliotecas

In [ ]:
import re
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, pipeline

***
# **Tratamento do dataset**
Nessa etapa, prepararemos o dataset em batches para ser passado ao modelo, adicionaremos tokens especiais e tokenizaremos as sentenças.   
Funções de tratamento de texto como remover caracteres indevidos devem ser aplicadas nessa fase

In [ ]:
df = pd.read_csv('/content/tripadvisor_hotel_reviews.csv')

In [ ]:
# Pegamos a média de palavras por amostra no dataset.
def get_num_words(df):
  num_words = []
  for id, row in df.iterrows():
    num_words.append(len(row['Review'].split()))
  
  num_words = np.array(num_words)
  return np.mean(num_words)

In [ ]:
MEAN_WORDS = get_num_words(df)

Para controlar nossa geração, utilizaremos os conceitos do paper [CTRL](https://arxiv.org/pdf/1909.05858.pdf). Nele, aprendemos a utilizar tokens de maneira a guiar o estilo de texto a ser gerado pelo modelo.   

Nesse caso, quero poder selecionar entre gerar analises boas e ruins.   

Nosso dataset possui um desbalanceamento claro entre as classes, o que poderia ser resolvido com técnicas de amostragem ou augmentação, mas como não é o foco desse notebook, apenas aplicarei o treinamento.

In [ ]:
df.Rating.value_counts()

5    9054
4    6039
3    2184
2    1793
1    1421
Name: Rating, dtype: int64

In [ ]:
def adicionar_tokens(row):
  # Os tokens serão inseridos dentro do texto e especificaremos eles ao utilizar o tokenizador.
  text = row['Review'] 
  if row['Rating'] >= 4:
    # Repare a necessidade do espaço, visto que caso esse não seja inserido, um token de classificação 
    # será misturado com uma palavra do texto

    # PS: O token de início de texto não é essencial.
    row['Review'] = "<|startoftext|> <BOM> " + text + ' <|endoftext|> '
  else:
    row['Review'] = "<|startoftext|> <RUIM> " + text + ' <|endoftext|> '
  
  return row


In [ ]:
# Aplicando o tratamento:
df = df.apply(adicionar_tokens, axis = 1)

In [ ]:
# Um exemplo de como fica nossa amostra: 
print(df.loc[0]['Review'])

<|startoftext|> <BOM> nice hotel expensive parking got good deal stay hotel anniversary, arrived late evening took advice previous reviews did valet parking, check quick easy, little disappointed non-existent view room room clean nice size, bed comfortable woke stiff neck high pillows, not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway, maybe just noisy neighbors, aveda bath products nice, did not goldfish stay nice touch taken advantage staying longer, location great walking distance shopping, overall nice experience having pay 40 parking night,   <|endoftext|> 


In [ ]:
# Separaremos nosso dataset em treino e validação da seguinte maneira:
train_set, val_set = train_test_split(df['Review'], test_size = 0.1, random_state = 42)


Salvaremos os conjuntos como CSV para carregá-los atraves da função load_dataset, que achei mais pratica utilizando arquivos CSVs,
mas que podem ser utilizados arquivos nos formatos JSON, Pandas e txt. [Documentação](https://huggingface.co/docs/datasets/)

In [ ]:
train_set.to_csv('train.csv')
val_set.to_csv('test.csv')

In [ ]:
dataset = load_dataset('csv', data_files =  {'train': 'train.csv',
                                             'test' : 'test.csv'});

Using custom data configuration default-de075c62ca943650


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-de075c62ca943650/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Review'],
        num_rows: 18441
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Review'],
        num_rows: 2050
    })
})

Agora já temos nosso DatasetDict. Temos uma coluna que não será utilizada, mas removeremos na próxima sessão, junto à tokenização.

***

# **Tokenizador**
O tokenizador converterá nossas palavras para números e gerará attention weights, que conduzirão o modelo na identificação de qual parte do contexto passado é relevante para a palavra atual.   

Utilizaremos o tokenizador pré-treinado da distilGPT2, que é um modelo menor e consequentemente mais rápido disponibilizado pela biblioteca.

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2', bos_token = '<|startoftext|>')

Adicionaremos nossos tokens especiais ao modelo.   
Os tokens especiais são guardados de maneira diferenciada pelo modelo, garantindo assim que esses não sofram cortes.   
Além disso, esses podem ser ocultados durante a geração. Mais detalhes na [documentação](https://huggingface.co/transformers/main_classes/tokenizer.html)

In [ ]:
# O valor de retorno 2 é referente à quantidade de tokens adicionados ao tokenizador.
tokenizer.add_special_tokens({'additional_special_tokens': ['<BOM>', '<RUIM>']})
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

0

In [ ]:
tokenizer.additional_special_tokens

['<BOM>', '<RUIM>']

In [ ]:
print("String antes da tokenização: \n")
print(df.loc[0]['Review'])

print('\nString após tokenização: \n')
print(tokenizer(df.loc[0]['Review']))

String antes da tokenização: 

<|startoftext|> <BOM> nice hotel expensive parking got good deal stay hotel anniversary, arrived late evening took advice previous reviews did valet parking, check quick easy, little disappointed non-existent view room room clean nice size, bed comfortable woke stiff neck high pillows, not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway, maybe just noisy neighbors, aveda bath products nice, did not goldfish stay nice touch taken advantage staying longer, location great walking distance shopping, overall nice experience having pay 40 parking night,   <|endoftext|> 

String após tokenização: 

{'input_ids': [50257, 50258, 44460, 7541, 5789, 7647, 1392, 922, 1730, 2652, 7541, 11162, 11, 5284, 2739, 6180, 1718, 5608, 2180, 8088, 750, 1188, 316, 7647, 11, 2198, 2068, 2562, 11, 1310, 11679, 1729, 12, 32786, 1570, 2119, 2119, 3424, 3621, 2546, 11, 3996, 6792, 19092, 15175, 7393, 1029, 9582, 1666, 11, 40

In [ ]:
tokenizer.bos_token_id

50257

## **Aplicando ao dataset**
Agora aplicaremos a tokenização linha por linha no nosso dataset de treino e validação.

In [ ]:
# Definiremos a função encode e aplicaremos o map nela, dividindo em batches a tokenização. 
# Depois removemos as colunas indesejadas

In [ ]:
def encode(data):
  return tokenizer(data['Review'], max_length= int(MEAN_WORDS), truncation = True, add_special_tokens= True, padding= False)

In [ ]:
train_set = dataset['train'].map(encode, batched = True).remove_columns(column_names= ['Unnamed: 0', 'Review']);
eval_set = dataset['test'].map(encode, batched = True).remove_columns(column_names= ['Unnamed: 0', 'Review']);

Utilizaremos o [Data collator](https://huggingface.co/transformers/main_classes/data_collator.html) para formar batches dos nossos itens de treinamento. 

In [ ]:
# O Parametro MLM é referente à masking language model e como utilizaremos a GPT2 sem MASKS, deixaremos desabilitado
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

In [ ]:
len(data_collator(train_set['input_ids']).get('input_ids'))

18441

***


#**Modelo** 
Uma vez que nossos dados e nosso tokenizador está pronto, inicializaremos o modelo e aplicaremos nosso fine-tuning.

In [ ]:
model = GPT2LMHeadModel.from_pretrained('distilgpt2', bos_token_id = tokenizer.bos_token_id);
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

In [ ]:
# Especificaremos parametros para o treinamento:
training_args = TrainingArguments(output_dir= 'GPTAIL/', overwrite_output_dir= True, 
                                  num_train_epochs = 5,
                                  save_steps= -1, per_device_train_batch_size = 8, 
                                  prediction_loss_only=True,
                                  per_device_eval_batch_size = 16, evaluation_strategy = 'epoch', load_best_model_at_end =True,
                                  eval_accumulation_steps = 1)


In [ ]:
trainer = Trainer(model = model, args = training_args, 
                train_dataset= train_set, eval_dataset=eval_set, data_collator = data_collator)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.989300,4.835339
2,4.756500,4.715217
3,4.653300,4.663646
4,4.559200,4.640491
5,4.517500,4.636616


TrainOutput(global_step=11530, training_loss=4.7500711895958405, metrics={'train_runtime': 2216.4855, 'train_samples_per_second': 5.202, 'total_flos': 0, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -351977472, 'train_mem_gpu_alloc_delta': 990287872, 'train_mem_cpu_peaked_delta': 364670976, 'train_mem_gpu_peaked_delta': 1419818496})

102.51406411049346

*** 
# **Geração**
Para a geração, utilizaremos a classe [Pipeline](https://huggingface.co/transformers/main_classes/pipelines.html), que une as principais tarefas em PLN. Nela selecionaremos a geração de texto e a configuraremos.

In [ ]:
model = model.to('cpu')

In [ ]:
revisor = pipeline('text-generation', tokenizer= tokenizer, model = model)

Usaremos o método de decodificação chamado Greedy Search. Nele, simplesmente pegaremos as palavras mais prováveis a cada etapa. O método influencia diretamente no resultado do modelo, portanto vale a pena dar uma olhada nos estilos de decodificação. [How to generate](https://huggingface.co/blog/how-to-generate)

In [ ]:
print(revisor("<BOM>", top_k = 30)[0].get('generated_text'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<BOM>good bad, arrived 3pm late october 2007 hotel beautiful clean comfortable good location, room small comfortable clean, view bay view ocean floor fabulous, bathroom pretty tiny clean, location great just blocks ginza beach restaurants shops restaurants,   


In [ ]:
print(revisor("<RUIM>", top_k = 30)[0].get('generated_text'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<RUIM>avoidance price paid stay, rooms small bathrooms poorly-maintained, bed hard pillows slippers no pillows, n't stay unless paid hotel,                


A geração possui diversos parametros a serem adaptados de acordo com o problema de vocês. Para mais detalhes: [Documentação](https://huggingface.co/transformers/main_classes/model.html?highlight=generate)
